In [131]:
import pandas as pd
import matplotlib.pyplot as plt
import os


In [ ]:
# database curation



In [16]:
# biorisk database

# read in regulated organisms
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import taxoniq # this is a new package which looks good, but has some issues with missing taxon IDs

# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('../client-secret.json', scope)
client = gspread.authorize(creds)

reg_taxons = []

# read in bacteria
sheet = client.open("Pathogen lists").worksheet("Bacteria")
bact_list = sheet.col_values(1)
australia = sheet.col_values(4)
us_select = sheet.col_values(5)
eu_export = sheet.col_values(7)
russia = sheet.col_values(8)

# add taxon ids to regulated list
for i in range(1,len(bact_list)):
    if ('Y' in [australia[i], us_select[i], eu_export[i], russia[i]]):
        reg_taxons.append(bact_list[i])

# read in viruses
sheet = client.open("Pathogen lists").worksheet("Viruses")
virus_list = sheet.col_values(1)
australia = sheet.col_values(4)
us_select = sheet.col_values(5)
eu_export = sheet.col_values(7)
russia = sheet.col_values(8)

# add taxon ids to regulated list
for i in range(1,len(virus_list)):
    if ('Y' in [australia[i], us_select[i], eu_export[i], russia[i]]):
        reg_taxons.append(virus_list[i])

# reg_taxons

In [118]:
# filter VFDB FASTA file

from Bio import SeqIO
import re

vfdb = pd.read_csv('databases/biorisk/VFs.csv')
vfdb.head()

regged = vfdb[vfdb['Bacteria'].isin(reg_taxons)]
regged

fetch = regged['VFID']

input_file = "databases/biorisk/VFDB_setA_pro.fas"
output_file = "databases/biorisk/VFDB_A_reg_filt_pro.fasta"

fasta = SeqIO.parse(input_file, "fasta")
records = ()

for r in fasta:
    if bool(re.search('\(VF\d+\)', r.description)):
        vfid = re.findall('\(VF\d+\)', r.description)[0]
        vfid = vfid[0].replace('(', '').replace(')', '')
        if fetch.str.contains(vfid).any():
            records = records + (r,)
SeqIO.write(records, output_file, "fasta")


In [222]:
# filter Victors FASTA file

from Bio import SeqIO
import re

def sequence_fetcher(input_file, output_file, reg_taxons):
    fasta = SeqIO.parse(input_file, "fasta")
    records = ()
    for r in fasta:
        for tax in reg_taxons:
            if bool(re.search(tax, r.description)):
                records = records + (r,)
    SeqIO.write(records, output_file, "fasta")

input_file = "databases/biorisk/victors_nt.fasta"
output_file = "databases/biorisk/victors_nt_reg_filt.fasta"

sequence_fetcher(input_file, output_file, reg_taxons)

input_file = "databases/biorisk/victors_prot.fasta"
output_file = "databases/biorisk/victors_prot_reg_filt.fasta"

sequence_fetcher(input_file, output_file, reg_taxons)

input_file = "databases/biorisk/VFDB_setA_pro.fasta"
output_file = "databases/biorisk/VFDB_setA_pro_reg_filt.fasta"

sequence_fetcher(input_file, output_file, reg_taxons)

input_file = "databases/biorisk/VFDB_setA_nt.fasta"
output_file = "databases/biorisk/VFDB_setA_nt_reg_filt.fasta"

sequence_fetcher(input_file, output_file, reg_taxons)

input_file = "databases/biorisk/VFDB_setB_pro.fasta"
output_file = "databases/biorisk/VFDB_setB_pro_reg_filt.fasta"

sequence_fetcher(input_file, output_file, reg_taxons)

input_file = "databases/biorisk/VFDB_setB_nt.fasta"
output_file = "databases/biorisk/VFDB_setB_nt_reg_filt.fasta"

sequence_fetcher(input_file, output_file, reg_taxons)


In [154]:
# benign database

cog_freq = pd.read_csv("databases/benign/STRING_COG_frequencies", header=None, sep=r"\s+")
cog_freq.head()

plt.figure(figsize=[10,6])
# n, bins, patches = plt.hist(x=cog_freq[0], bins=50, color='#0504aa',alpha=0.7, rwidth=0.85, range=(0, 1000))
n, bins, patches = plt.hist(x=cog_freq[0], bins=50, color='#0504aa',alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value',fontsize=15)
plt.ylabel('Frequency',fontsize=15)
plt.show()

common_genes = cog_freq[1][cog_freq[0]>1000]
print(len(common_genes))

for i in common_genes:
    os.system("cat databases/benign/2/" + i + ".hmm.gz >> databases/benign/common_STRING.hmm")    
    os.system("grep " + i + " databases/benign/2_annotations.tsv >> databases/benign/common_STRING_annotations.tsv")

In [216]:
# freegenes Genbanks to FASTA

from Bio import SeqIO
from Bio import SeqUtils
from Bio import Seq
import glob
gbks = glob.glob("databases/benign/freegenes.github.io/genbank/*.gb_fixed")

for i in gbks:
    gbk_filename = i
    faa_filename = i.replace("genbank", "fasta").replace(".gb_fixed", ".fasta")
    input_handle  = open(gbk_filename, "r")
    output_handle = open(faa_filename, "w")
    
    for seq_record in SeqIO.parse(input_handle, "gb"):
        output_handle.write(">%s %s\n%s\n" % (
           i.replace("databases/benign/freegenes.github.io/genbank/", "").replace(".gb_fixed", ""),
           seq_record.description,
           seq_record.seq))
    
    output_handle.close()
    input_handle.close()

BBF10K_000184
synthetic linear DNA
ATGAGCAATTTCAAATCGGGCTTTGTTAGCATCATTGGCCGCCCGAATGTGGGGAAAAGCACCTTATTAAACAAGTTAATTGGCGAGAAGATTTCGATTGTCACCAACAAGCCACAGACCACACGCAATAATATTCGCGGCATCTTGACGAAGAAGGATCAATACCAGATCGTGTTCATTGATACACCAGGAGTCCACACAAGTAAAAAGCAATTGGACAAATTTCTGAATACGTCGGCACTGAAATCAACGAAGGACGTAGACGTAATCTTGTTCTTGGCGCCCTCTGACGAAGTTATCGGCAAGAATGACTTGTTTCTGTTAAAGCAAATCGAAAATCTGGACGTGTTCAAAATCCTTGTCATTACAAAAGCGGATAGCGTCACCAAAGAACAGCTGATCTTAAAGGCGAACGAGTGGTCATCATACCAGGACCAGTTCGATGAAATCATTATTACTTCATCCATCACTAACCTTAACATTGAGAAACTGTTAGAGCTGATTGTAAACAATCTGTCAGATAATGATTATCAATTTTACGACGACGATATCCTGACCGACCAATCTGACCGTTTCATGATCAAAGAGATTATCCGTGAAAACATTTTGTTAAAGACCGGGCAGGAGGTCCCTCACTCCGTTGCAGTTTTGGTCGAGCATCTTGAACAGAACGAAAACGAGATCAATATCTCGTGTGCTATTATTGTGGAACGTCAAAGCCAGAAAAGCATTATTATCGGAAAAAATGGCGTTAAAATCTCTGACATTCGCTATAAAAGTAAAAAGCAAATTCAGGCACTGTTCAAAAAGCGCATCAATCTTGAGTTGTTTGTTAAAGTGCAGGAAAATTGGCGCAATTCGCCGAGTCTGATTAAGAAGATGGGATATAATAAGGACCAATACTGAAGAGCTTAGAGACCGGTACTAGTTCCTGCAGGGCTCGAGGTGTCAATCGTCGGAGCCGC

ValueError: Problem with 'CDS' feature:
29..2344
/label="Pyrococcus Sp. Heat-Stable DNA Polymerase

In [212]:
input_handle = open(gbks[2], "r")

for seq_record in SeqIO.parse(input_handle, "gb"):
    print(gbks[1].replace("databases/benign/freegenes.github.io/genbank/", "").replace(".gb_fixed", "")),
    print(seq_record.id)
    print(seq_record.seq)

BBF10K_000185
.
ACAACGAGCAGACCGAATAGGGTCTCAAATGAACAGCAACATTAAGGCCATCGTCGAAGGTCTGCTGTTCGTGTACGGTGATGATGGTATTTCCCTGTTGGATCTGCAGAATGTGCTGGAGGACGTTCGTCCAGTTACGATCCAGGAGGCGATCCTGGAGCTGGAGAAAAAGTACAGCAGCGACCCGGATTGCGCATTCAGCATCCAGAAGTTTGGCAAGAACAAATACCGCTTACAGACCAAGCCTGAACTGCACGAGTACTTTGCGAAATTGGAGCTGGAAGTTAACAATAGCCGTCTGAGCAATAGCAGCATTGAGGTGCTGAGCATCATCGTCTATAAAGGTCCGATTAGCAAGCACGATATTGAGCTGATTCGTCAAGCGGAGTGTAGCTACCAGATCTACCGTCTGCGTCAGAAGAAGTTGATTAAGGCAGTTGGCAAAACCAGCACCGGCGCGAATCTGTATACCATCACCGATAACTTTTTCAAACTCTTTAACATTACCGGCGGTTTTGAGGCACTGCCACAAATTGACTTCGCGAGCTACAAGAATCAGGATGAAGAGAATAATTTCGAAGAGGAGATCAAAGTCACCGAGGAGATGATCTCCGAGGAGGACGTGTTTGAAGAGGACGACAGCGAAGGCATGTTTTGAAGAGCTTAGAGACCCGCGGAAAATGGAAAACACTCGCTGAGGTGTCAATCGTCGGAGCCGCTGAGCAATAACTAGCATAACCCCTTGGGGCCTCTAAACGGGTCTTGAGGGGTTTTTTGCATGGTCATAGCTGTTTCCTGAGAGCTTGGCAGGTGATGACACACATTAACAAATTTCGTGAGGAGTCTCCAGAAGAATGCCATTAATTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCG

In [202]:
gbks[1]

'databases/benign/freegenes.github.io/genbank/BBF10K_003444.gb'